In [1]:
import time
import random
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, MaxPooling2D, Conv2D, Dropout, Lambda, Dense, Flatten, Activation, Input, Embedding, BatchNormalization
from tensorflow.keras.initializers import glorot_normal, Zeros, TruncatedNormal
from tensorflow.keras.regularizers import l2


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy


from tensorflow.keras.optimizers import Adam
from collections import defaultdict
import math

In [2]:
class FeaturesEmbedding(Layer):  
    '''
    임베딩 레이어입니다. 
    - 만약 피처(feature) 3개가 각각 10개, 20개, 30개의 고유값을 가진다면 feature_dims는 [10, 20, 30] 형태를 띄게 됩니다.
    - 전체 임베딩을 해야 할 개수는 10+20+30 = 60이므로 '60 x 임베딩_차원_크기'의 행렬이 생성되게 됩니다.
    '''
    def __init__(self, field_dims, embed_dim, **kwargs):
        super(FeaturesEmbedding, self).__init__(**kwargs)
        self.total_dim = sum(field_dims)
        self.embed_dim = embed_dim
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.longlong)
        self.embedding = tf.keras.layers.Embedding(input_dim=self.total_dim, output_dim=self.embed_dim)

    def build(self, input_shape):
        # 임베딩을 빌드하고 초기화합니다.
        self.embedding.build(input_shape)
        self.embedding.set_weights([tf.keras.initializers.GlorotUniform()(shape=self.embedding.weights[0].shape)])

    def call(self, x):
        # 들어온 입력의 임베딩을 가져니다.
        x = x + tf.constant(self.offsets)
        return self.embedding(x)

In [3]:
class FeaturesEmbedding(Layer):  
    '''
    임베딩 레이어입니다. 
    - 만약 피처(feature) 3개가 각각 10개, 20개, 30개의 고유값을 가진다면 feature_dims는 [10, 20, 30] 형태를 띄게 됩니다.
    - 전체 임베딩을 해야 할 개수는 10+20+30 = 60이므로 '60 x 임베딩_차원_크기'의 행렬이 생성되게 됩니다.
    '''
    def __init__(self, field_dims, embed_dim, **kwargs):
        super(FeaturesEmbedding, self).__init__(**kwargs)
        self.total_dim = sum(field_dims)
        self.embed_dim = embed_dim
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.longlong)
        self.embedding = tf.keras.layers.Embedding(input_dim=self.total_dim, output_dim=self.embed_dim)

    def build(self, input_shape):
        # 임베딩을 빌드하고 초기화합니다.
        self.embedding.build(input_shape)
        self.embedding.set_weights([tf.keras.initializers.GlorotUniform()(shape=self.embedding.weights[0].shape)])

    def call(self, x):
        # 들어온 입력의 임베딩을 가져니다.
        x = x + tf.constant(self.offsets)
        return self.embedding(x)
class MultiLayerPerceptron(Layer):  
    '''
    DNN 레이어입니다.
    - Tensorflow Keras에서는 Dense 레이어를 쌓아올린 구조입니다.
    - 필요에 따라 배치 정규화도 사용할 수 있습니다.
    '''
    def __init__(self, input_dim, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False, init_std=0.0001, output_layer=True):
        super(MultiLayerPerceptron, self).__init__()
        self.dropout_rate = dropout_rate
        self.use_bn = use_bn
        hidden_units = [input_dim] + list(hidden_units)
        if output_layer:
            hidden_units += [1]
        # Dense layer를 쌓아올립니다.
        self.linears = [Dense(units, activation=None, kernel_initializer=tf.random_normal_initializer(stddev=init_std),
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg)) for units in hidden_units[1:]]
        # 활성화 함수를 세팅합니다.
        self.activation = tf.keras.layers.Activation(activation)
        # 필요하다면 배치정규화도 진행합니다.
        if self.use_bn:
            self.bn = [BatchNormalization() for _ in hidden_units[1:]]
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        x = inputs
        for i in range(len(self.linears)):
            # input data가 들어오면 layer를 돌면서 벡터 값을 가져오게 됩니다.
            x = self.linears[i](x)
            if self.use_bn:
                x = self.bn[i](x, training=training)
            # 각 layer마다 나온 벡터 값에 활성화 함수와 dropout을 적용시켜 비선형성 구조와 과적합을 방지합니다.
            x = self.activation(x)
            x = self.dropout

In [4]:
class MultiHeadSelfAttention(Layer):  
    '''
    멀티 헤드 셀프 어텐션 레이어입니다.
    - 위에 작성한 수식과 같이 동작됩니다.
    - 필요에 따라 잔차 연결(residual connection)도 진행합니다.
    '''
    def __init__(self, att_embedding_size=8, head_num=2, use_res=True, scaling=False, seed=1024, **kwargs):
        if head_num <= 0:
            raise ValueError('head_num must be a int > 0')
        self.att_embedding_size = att_embedding_size
        self.head_num = head_num
        self.use_res = use_res
        self.seed = seed
        self.scaling = scaling
        super(MultiHeadSelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (len(input_shape)))
        embedding_size = int(input_shape[-1])
        # 쿼리에 해당하는 매트릭스입니다. 
        self.W_Query = self.add_weight(name='query', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed))
        # 키에 해당되는 매트릭스입니다.
        self.W_key = self.add_weight(name='key', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                     dtype=tf.float32,
                                     initializer=TruncatedNormal(seed=self.seed + 1))
        # 값(value)에 해당되는 매트릭스입니다.
        self.W_Value = self.add_weight(name='value', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed + 2))
        # 필요하다면 잔차 연결도 할 수 있습니다.
        if self.use_res:
            self.W_Res = self.add_weight(name='res', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                         dtype=tf.float32,
                                         initializer=TruncatedNormal(seed=self.seed))

        super(MultiHeadSelfAttention, self).build(input_shape)

    def call(self, inputs, **kwargs):
        if K.ndim(inputs) != 3:
            raise ValueError("Unexpected inputs dimensions %d, expect to be 3 dimensions" % (K.ndim(inputs)))
        
        # 입력이 들어오면 쿼리, 키, 값(value)에 매칭되어 각각의 값을 가지고 옵니다.
        querys = tf.tensordot(inputs, self.W_Query, axes=(-1, 0))  
        keys = tf.tensordot(inputs, self.W_key, axes=(-1, 0))
        values = tf.tensordot(inputs, self.W_Value, axes=(-1, 0))

        # 헤드 개수에 따라 데이터를 분리해줍니다.
        querys = tf.stack(tf.split(querys, self.head_num, axis=2))
        keys = tf.stack(tf.split(keys, self.head_num, axis=2))
        values = tf.stack(tf.split(values, self.head_num, axis=2))
        
        # 쿼리와 키를 먼저 곱해줍니다. 위 이미지의 식 (5)와 같습니다.
        inner_product = tf.matmul(querys, keys, transpose_b=True)
        if self.scaling:
            inner_product /= self.att_embedding_size ** 0.5
        self.normalized_att_scores =  tf.nn.softmax(inner_product)
        
        # 쿼리와 키에서 나온 어텐션 값을 값(value)에 곱해줍니다. 식 (6)과 같습니다.
        result = tf.matmul(self.normalized_att_scores, values)
        # 식 (7)과 같이 쪼개어진 멀테 헤드를 모아줍니다.
        result = tf.concat(tf.split(result, self.head_num, ), axis=-1)
        result = tf.squeeze(result, axis=0) 

        if self.use_res:
            result += tf.tensordot(inputs, self.W_Res, axes=(-1, 0))
        result = tf.nn.relu(result)
        
        # 그 결과 값을 리턴합니다.

        return result

    def compute_output_shape(self, input_shape):

        return (None, input_shape[1], self.att_embedding_size * self.head_num)

    def get_config(self, ):
        config = {'att_embedding_size': self.att_embedding_size, 'head_num': self.head_num, 'use_res': self.use_res,'seed': self.seed}
        base_config = super(MultiHeadSelfAttention, self).get_config()
        base_config.update(config)
        return base_config

## 바꿀 부분
- 밑의 코드의 모델 정의 부분을 새로운 모델로 해줘야된다. 

In [5]:
class AutoIntMLP(Layer): 
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2, att_res=True, dnn_hidden_units=(32, 32), dnn_activation='relu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False, dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLP, self).__init__()
        self.embedding = FeaturesEmbedding(field_dims, embedding_size)
        self.num_fields = len(field_dims)
        self.embedding_size = embedding_size

        self.final_layer = Dense(1, use_bias=False, kernel_initializer=tf.random_normal_initializer(stddev=init_std))
        
        self.dnn = tf.keras.Sequential()
        for units in dnn_hidden_units:
            self.dnn.add(Dense(units, activation=dnn_activation,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_reg_dnn),
                               kernel_initializer=tf.random_normal_initializer(stddev=init_std)))
            if dnn_use_bn:
                self.dnn.add(BatchNormalization())
            self.dnn.add(Activation(dnn_activation))
            if dnn_dropout > 0:
                self.dnn.add(Dropout(dnn_dropout))
        self.dnn.add(Dense(1, kernel_initializer=tf.random_normal_initializer(stddev=init_std)))

        self.int_layers = [MultiHeadSelfAttention(att_embedding_size=embedding_size, head_num=att_head_num, use_res=att_res) for _ in range(att_layer_num)]

    def call(self, inputs):
        embed_x = self.embedding(inputs)
        dnn_embed = tf.reshape(embed_x, shape=(-1, self.embedding_size * self.num_fields))

        att_input = embed_x
        for layer in self.int_layers:
            att_input = layer(att_input)

        att_output = Flatten()(att_input)
        att_output = self.final_layer(att_output)
        
        dnn_output = self.dnn(dnn_embed)
        y_pred = tf.keras.activations.sigmoid(att_output + dnn_output)
        
        return y_pred

In [6]:
def get_DCG(ranklist, y_true):
    dcg = 0.0
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item in y_true:
            dcg += 1.0 / math.log(i + 2)
    return  dcg

def get_IDCG(ranklist, y_true):
    idcg = 0.0
    i = 0
    for item in y_true:
        if item in ranklist:
            idcg += 1.0 / math.log(i + 2)
            i += 1
    return idcg

def get_NDCG(ranklist, y_true):
    '''NDCG 평가 지표'''
    ranklist = np.array(ranklist).astype(int)
    y_true = np.array(y_true).astype(int)
    dcg = get_DCG(ranklist, y_true)
    idcg = get_IDCG(y_true, y_true)
    if idcg == 0:
        return 0
    return round( (dcg / idcg), 5)

def get_hit_rate(ranklist, y_true):
    '''hitrate 평가 지표'''
    c = 0
    for y in y_true:
        if y in ranklist:
            c += 1
    return round( c / len(y_true), 5 )

In [7]:
def test_model(model, test_df):
    '''모델 테스트'''
    user_pred_info = defaultdict(list)
    total_rows = len(test_df)
    for i in range(0, total_rows, batch_size):
        features = test_df.iloc[i:i + batch_size, :-1].values
        y_pred = model.predict(features, verbose=False)
        for feature, p in zip(features, y_pred):
            u_i = feature[:2]
            user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
    return user_pred_info

## 학습용 데이터 만들기

In [8]:
project_path = os.path.abspath(os.getcwd())
data_dir_nm = 'data'
movielens_dir_nm = 'ml-1m'
model_dir_nm = 'model'
data_path = f"{project_path}/{data_dir_nm}"

In [9]:
# 1. 데이터 불러오기
# csv 데이터이므로 read_csv로 가져옵니다.
movielens_rcmm = pd.read_csv(f"{data_path}/ml-1m/movielens_rcmm_v2.csv", dtype=str)
print(movielens_rcmm.shape)
movielens_rcmm.head()

(1000209, 15)


,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,rating_decade,genre1,genre2,genre3,gender,age,occupation,zip,label
0,1,1193,1970s,1975,2001,1,2000s,Drama,no,no,F,1,10,48067,1
1,1,661,1990s,1996,2001,1,2000s,Animation,Children's,Musical,F,1,10,48067,0
2,1,914,1960s,1964,2001,1,2000s,Musical,Romance,no,F,1,10,48067,0
3,1,3408,2000s,2000,2001,1,2000s,Drama,no,no,F,1,10,48067,1
4,1,2355,1990s,1998,2001,1,2000s,Animation,Children's,Comedy,F,1,10,48067,1


In [10]:
label_encoders = {col: LabelEncoder() for col in movielens_rcmm.columns[:-1]} # label은 제외

for col, le in label_encoders.items():
    movielens_rcmm[col] = le.fit_transform(movielens_rcmm[col])

In [11]:
movielens_rcmm.head()

,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,rating_decade,genre1,genre2,genre3,gender,age,occupation,zip,label
0,0,189,6,55,1,0,0,7,17,15,0,0,2,1588,1
1,0,3374,8,76,1,0,0,2,2,8,0,0,2,1588,0
2,0,3615,5,44,1,0,0,11,12,15,0,0,2,1588,0
3,0,2503,9,80,1,0,0,7,17,15,0,0,2,1588,1
4,0,1374,8,78,1,0,0,2,2,2,0,0,2,1588,1


In [12]:
movielens_rcmm['label'] = movielens_rcmm['label'].astype(np.float32)

In [13]:
# 3. 학습 데이터와 테스트데이터로 분리, 0.2 정도로 분리
train_df, test_df = train_test_split(movielens_rcmm, test_size=0.2, random_state=42)

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800167 entries, 416292 to 121958
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   user_id        800167 non-null  int64  
 1   movie_id       800167 non-null  int64  
 2   movie_decade   800167 non-null  int64  
 3   movie_year     800167 non-null  int64  
 4   rating_year    800167 non-null  int64  
 5   rating_month   800167 non-null  int64  
 6   rating_decade  800167 non-null  int64  
 7   genre1         800167 non-null  int64  
 8   genre2         800167 non-null  int64  
 9   genre3         800167 non-null  int64  
 10  gender         800167 non-null  int64  
 11  age            800167 non-null  int64  
 12  occupation     800167 non-null  int64  
 13  zip            800167 non-null  int64  
 14  label          800167 non-null  float32
dtypes: float32(1), int64(14)
memory usage: 94.6 MB


In [15]:
# 필요 컬럼들과 레이블 정의
# 필드의 각 고유 개수를 정의하는 field_dims를 정의합니다. 이는  임베딩 때 활용됩니다. 
u_i_feature = ['user_id', 'movie_id']
meta_features = ['movie_decade', 'movie_year', 'rating_year', 'rating_month', 'rating_decade', 'genre1','genre2', 'genre3', 'gender', 'age', 'occupation', 'zip']
label = 'label'
field_dims = np.max(movielens_rcmm[u_i_feature + meta_features].astype(np.int64).values, axis=0) + 1
field_dims

array([6040, 3706,   10,   81,    4,   12,    1,   18,   18,   16,    2,
          7,   21, 3439])

In [16]:
# 에포크, 학습률, 드롭아웃, 배치사이즈, 임베딩 크기 등 정의
epochs=5
learning_rate= 0.0001
dropout= 0.4
batch_size = 2048
embed_dim= 16

## 고칠부분
- 밑의 코드에서도 AutoIntMLP를 가져오고, DNN 레이어가 붙은 파라미터를 추가해준다.

In [17]:
# AutoInt 레이어를 가지고 있는 모델 본체입니다. 해당 모델을 활용해 훈련을 진행합니다.
class AutoIntModel(Model):
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2,
                 att_res=True, dnn_hidden_units=(32, 32), dnn_activation='relu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=False,
                 dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntModel, self).__init__()
        self.autoInt_layer = AutoIntMLP(
            field_dims=field_dims,
            embedding_size=embedding_size,
            att_layer_num=att_layer_num,
            att_head_num=att_head_num,
            att_res=att_res,
            dnn_hidden_units=dnn_hidden_units,
            dnn_activation=dnn_activation,
            l2_reg_dnn=l2_reg_dnn,
            l2_reg_embedding=l2_reg_embedding,
            dnn_use_bn=dnn_use_bn,
            dnn_dropout=dnn_dropout,
            init_std=init_std
        )

    def call(self, inputs, training=False):
        return self.autoInt_layer(inputs, training=training)

In [18]:
# 모델 정의
autoInt_model = AutoIntModel(
    field_dims=field_dims,
    embedding_size=embed_dim,
    att_layer_num=3,
    att_head_num=2,
    att_res=True,
    dnn_hidden_units=(32, 32),               # 추가: DNN 은닉층 구조
    dnn_activation='relu',                  # 추가: 활성화 함수
    l2_reg_dnn=0,
    l2_reg_embedding=1e-5,
    dnn_use_bn=False,
    dnn_dropout=dropout,
    init_std=0.0001
)

In [19]:
# 옵티마이저, 오차함수 정의
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy(from_logits=False)

In [20]:
autoInt_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])

In [21]:
history = autoInt_model.fit(train_df[u_i_feature + meta_features], train_df[label], epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 128s 321ms/step - binary_crossentropy: 0.6873 - loss: 0.6873 - val_binary_crossentropy: 0.6494 - val_loss: 0.6494
Epoch 2/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 118s 334ms/step - binary_crossentropy: 0.6347 - loss: 0.6347 - val_binary_crossentropy: 0.5889 - val_loss: 0.5889
Epoch 3/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 133s 378ms/step - binary_crossentropy: 0.5699 - loss: 0.5699 - val_binary_crossentropy: 0.5484 - val_loss: 0.5484
Epoch 4/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 133s 351ms/step - binary_crossentropy: 0.5435 - loss: 0.5435 - val_binary_crossentropy: 0.5433 - val_loss: 0.5433
Epoch 5/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 107s 304ms/step - binary_crossentropy: 0.5369 - loss: 0.5369 - val_binary_crossentropy: 0.5418 - val_loss: 0.5418


In [29]:
# 사용자에게 예측된 정보를 저장하는 딕셔너리 
user_pred_info = {}
# top10개
top = 10
# 테스트 값을 가지고 옵니다. 
mymodel_user_pred_info = test_model(autoInt_model, test_df)
# 사용자마다 돌면서 예측 데이터 중 가장 높은 top 10만 가져옵니다. 
for user, data_info in tqdm(mymodel_user_pred_info.items(), total=len(mymodel_user_pred_info), position=0, leave=True):
    ranklist = sorted(data_info, key=lambda s : s[1], reverse=True)[:top]
    ranklist = list(dict.fromkeys([r[0] for r in ranklist]))
    user_pred_info[str(user)] = ranklist
# 원본 테스트 데이터에서 label이 1인 사용자 별 영화 정보를 가져옵니다.
test_data = test_df[test_df['label']==1].groupby('user_id')['movie_id'].apply(list)

C:\Users\lynn9\AppData\Local\Temp\ipykernel_4272\2567990266.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
100%|██████████| 6038/6038 [00:00<00:00, 35900.94it/s]


In [30]:
mymodel_ndcg_result = {}
mymodel_hitrate_result = {}

# 모델 예측값과 원본 테스트 데이터를 비교해서 어느정도 성능이 나왔는지 NDCG와 Hitrate를 비교합니다.

# NDCG
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # NDCG 값 구하기
    user_ndcg = get_NDCG(mymodel_pred, testset)

    mymodel_ndcg_result[user] = user_ndcg

# Hitrate
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # hitrate 값 구하기
    user_hitrate = get_hit_rate(mymodel_pred, testset)

    # 사용자 hitrate 결과 저장
    mymodel_hitrate_result[user] = user_hitrate

  0%|          | 0/5994 [00:00<?, ?it/s]

100%|██████████| 5994/5994 [00:00<00:00, 23525.95it/s]


In [31]:
print(" mymodel ndcg : ", round(np.mean(list(mymodel_ndcg_result.values())), 5))
print(" mymodel hitrate : ", round(np.mean(list(mymodel_hitrate_result.values())), 5))

 mymodel ndcg :  0.66202
 mymodel hitrate :  0.63058


In [32]:
autoInt_model.save_weights('./model/autoInt_model_weights.weights.h5')

-----------------------------------------------------------------------------------

### [AutoInt+ 모델 성능 개선 시도 -> 실험 2가지]
- 성능 개선은 총 2가지 방법으로 시도했다. -> 근데 2가지 방법 모두 실패 ㅋㅋ.. ㅠ
- try1)
    - 위 코드 (기존 Baseline)의 구조는 그대로 유지하되, 모델 용량을 확장해봤다.
    -> att_head_num: 2 → 4 // dnn_hidden_units: (32, 32) → (64, 32, 16) (DNN 계층 추가 및 확장) // dnn_use_bn: False → True (배치 정규화 적용) // dnn_dropout: 0.4 → 0.3 (드롭아웃 감소 → 과적합 완화 기대)
- try2) 
    - 구조는 그대로 유지하되, dnn_dropout을 0.4 -> 0.3으로 감소시켰다.
    -> 과도한 정규화로 인한 손실 방지를 위해서..

### try1

In [43]:
import itertools
import numpy as np
import pandas as pd
from collections import defaultdict
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
import tensorflow as tf

# 경로 추가해서 model import 가능하게 함
import sys
sys.path.append(r'C:\Users\lynn9\PythonDev\MyProject\aiffel\autoint\model')
from autointmlp import AutoIntMLPModel

In [44]:
# <평가함수 정의>
def get_DCG(ranklist, y_true):
    dcg = 0.0
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item in y_true:
            dcg += 1.0 / math.log(i + 2)
    return  dcg

def get_IDCG(ranklist, y_true):
    idcg = 0.0
    i = 0
    for item in y_true:
        if item in ranklist:
            idcg += 1.0 / math.log(i + 2)
            i += 1
    return idcg

def get_NDCG(ranklist, y_true):
    ranklist = np.array(ranklist).astype(int)
    y_true = np.array(y_true).astype(int)
    dcg = get_DCG(ranklist, y_true)
    idcg = get_IDCG(y_true, y_true)
    if idcg == 0:
        return 0
    return round((dcg / idcg), 5)

def get_hit_rate(ranklist, y_true):
    c = 0
    for y in y_true:
        if y in ranklist:
            c += 1
    return round(c / len(y_true), 5)

def test_model(model, test_df, batch_size=2048, top=10):
    user_pred_info = defaultdict(list)
    total_rows = len(test_df)
    for i in range(0, total_rows, batch_size):
        features = test_df.iloc[i:i + batch_size, :-1].values
        y_pred = model.predict(features, verbose=False)
        for feature, p in zip(features, y_pred):
            u_i = feature[:2]
            user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
    return user_pred_info

In [45]:
# <모델 예측 테스트 함수>
def test_model(model, test_df, batch_size=2048, top=10):
    user_pred_info = defaultdict(list)
    for i in range(0, len(test_df), batch_size):
        features = test_df.iloc[i:i + batch_size, :-1].values
        y_pred = model.predict(features, verbose=False)
        for feature, p in zip(features, y_pred):
            u_i = feature[:2]
            user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
    return user_pred_info

In [ ]:
# <데이터 로딩 및 전처리>
project_path = os.getcwd()
data_path = os.path.join(project_path, 'data', 'ml-1m')
csv_path = os.path.join(data_path, 'movielens_rcmm_v2.csv')

df = pd.read_csv(csv_path, dtype=str)
label_encoders = {col: LabelEncoder() for col in df.columns[:-1]}
for col, le in label_encoders.items():
    df[col] = le.fit_transform(df[col])
df['label'] = df['label'].astype(np.float32)

# 컬럼 분리
u_i_feature = ['user_id', 'movie_id']
meta_features = ['movie_decade', 'movie_year', 'rating_year', 'rating_month', 'rating_decade',
                 'genre1', 'genre2', 'genre3', 'gender', 'age', 'occupation', 'zip']
all_features = u_i_feature + meta_features
label = 'label'

field_dims = np.max(df[all_features].astype(np.int64).values, axis=0) + 1
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [47]:
# <모델 구성 및 학습 -> 기존 구조 유지 + 강화하는 방향>
autoIntMLP_model = AutoIntMLPModel(
    field_dims=field_dims,
    embedding_size=16,
    att_layer_num=3,
    att_head_num=4,                   # ← 헤드 수 증가
    att_res=True,
    dnn_hidden_units=(64, 32, 16),    # ← DNN 레이어 확장
    dnn_activation='relu',
    l2_reg_dnn=0,
    l2_reg_embedding=1e-5,
    dnn_use_bn=True,                  # ← 배치 정규화 사용
    dnn_dropout=0.3,                  # ← 드롭아웃 살짝 감소
    init_std=0.0001
)

optimizer = Adam(learning_rate=0.0001)
loss_fn = BinaryCrossentropy(from_logits=False)

autoIntMLP_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])

# 학습
autoIntMLP_model.fit(train_df[all_features], train_df[label],
                     epochs=5, batch_size=2048, validation_split=0.1, verbose=1)

Epoch 1/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 387s 1s/step - binary_crossentropy: 0.6799 - loss: 0.6799 - val_binary_crossentropy: 0.6467 - val_loss: 0.6467
Epoch 2/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 347s 988ms/step - binary_crossentropy: 0.5947 - loss: 0.5947 - val_binary_crossentropy: 0.5567 - val_loss: 0.5567
Epoch 3/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 579s 2s/step - binary_crossentropy: 0.5437 - loss: 0.5437 - val_binary_crossentropy: 0.5411 - val_loss: 0.5411
Epoch 4/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 465s 1s/step - binary_crossentropy: 0.5348 - loss: 0.5348 - val_binary_crossentropy: 0.5379 - val_loss: 0.5379
Epoch 5/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 510s 1s/step - binary_crossentropy: 0.5307 - loss: 0.5307 - val_binary_crossentropy: 0.5358 - val_loss: 0.5358


In [48]:
# <모델 평가>
user_pred_info = test_model(autoIntMLP_model, test_df, top=10)
test_data = test_df[test_df['label'] == 1].groupby('user_id')['movie_id'].apply(list)

ndcgs, hitrates = [], []
for user, true_items in tqdm(test_data.items(), desc='Evaluate'):
    pred = user_pred_info.get(int(user), [])[:10]
    pred_items = [x[0] for x in pred]
    ndcgs.append(get_NDCG(pred_items, true_items))
    hitrates.append(get_hit_rate(pred_items, true_items))

print(f"\nmymodel ndcg :  {round(np.mean(ndcgs), 5)}")
print(f"mymodel hitrate :  {round(np.mean(hitrates), 5)}")

C:\Users\lynn9\AppData\Local\Temp\ipykernel_4272\3984162195.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
Evaluate: 5994it [00:09, 616.70it/s]



mymodel ndcg :  0.55717
mymodel hitrate :  0.57051


### try2

In [49]:
# <모델 구성 및 학습 -> 기존 구조 유지 + 강화하는 방향>
# 아예 하이퍼파라미터 튜닝 전의 기존 설정에서 dropout만 조정함
epochs = 5
learning_rate = 0.0001
dropout = 0.3  # 기존 0.4 → 수정
batch_size = 2048
embed_dim = 16

# 모델 정의
autoIntMLP_model = AutoIntMLPModel(
    field_dims=field_dims,
    embedding_size=embed_dim,
    att_layer_num=3,
    att_head_num=2,
    att_res=True,
    dnn_hidden_units=(32, 32),
    dnn_activation='relu',
    l2_reg_dnn=0,
    l2_reg_embedding=1e-5,
    dnn_use_bn=False,
    dnn_dropout=dropout,
    init_std=0.0001
)

In [50]:
# 컴파일 및 학습
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy(from_logits=False)

autoIntMLP_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])

history = autoIntMLP_model.fit(
    train_df[u_i_feature + meta_features], 
    train_df[label],
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    verbose=1
)

Epoch 1/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 216s 598ms/step - binary_crossentropy: 0.6874 - loss: 0.6874 - val_binary_crossentropy: 0.6493 - val_loss: 0.6493
Epoch 2/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 195s 552ms/step - binary_crossentropy: 0.6342 - loss: 0.6342 - val_binary_crossentropy: 0.5864 - val_loss: 0.5864
Epoch 3/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 150s 424ms/step - binary_crossentropy: 0.5691 - loss: 0.5691 - val_binary_crossentropy: 0.5474 - val_loss: 0.5474
Epoch 4/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 193s 550ms/step - binary_crossentropy: 0.5424 - loss: 0.5424 - val_binary_crossentropy: 0.5428 - val_loss: 0.5428
Epoch 5/5
352/352 ━━━━━━━━━━━━━━━━━━━━ 144s 408ms/step - binary_crossentropy: 0.5377 - loss: 0.5377 - val_binary_crossentropy: 0.5408 - val_loss: 0.5408


In [51]:
# <모델 평가>
user_pred_info = test_model(autoIntMLP_model, test_df, top=10)
test_data = test_df[test_df['label'] == 1].groupby('user_id')['movie_id'].apply(list)

ndcgs, hitrates = [], []
for user, true_items in tqdm(test_data.items(), desc='Evaluate'):
    pred = user_pred_info.get(int(user), [])[:10]
    pred_items = [x[0] for x in pred]
    ndcgs.append(get_NDCG(pred_items, true_items))
    hitrates.append(get_hit_rate(pred_items, true_items))

print(f"\nmymodel ndcg :  {round(np.mean(ndcgs), 5)}")
print(f"mymodel hitrate :  {round(np.mean(hitrates), 5)}")

C:\Users\lynn9\AppData\Local\Temp\ipykernel_4272\3984162195.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
Evaluate: 5994it [00:00, 9393.00it/s]


mymodel ndcg :  0.55717
mymodel hitrate :  0.57051


-------------------------------------------------------------------------------------

### [AutoInt+ 모델 성능 개선 시도 - 실험 결과 및 정리]
- 기존 Baseline
    - NDCG: 0.66202, HitRate: 0.63058
    - 위의 평가지표는 AutoInt+ 모델이 feature 간의 복합적인 상호작용을 잘 학습했다고 판단할 수 있으며, 적절한 dropout(0.4)으로 정규화도 나름 효과적이라고 볼 수 있다..!
- try1)
    - 과도한 복잡성 도입과 과적합 억제 간 균형을 이루지 못해서 성능이 더 하락했다.
    - 내 예상과는 다르게, DNN 확장은 오히려 일반화 성능을 더 저하시켰다.
- try2) 
    - dropout만 0.4 -> 0.3으로 약간 줄였는데, 변화가 없었다..
    - 이는 아마도 단일 하이퍼 파라미터를 변경하는 것만으로는 성능의 개선이 어렵다고 판단할 수 있다.
- 결론: Baseline 대비 2개 실험 모두 성능이 떨어졌다. 조금 더 성능 개선의 방법을 찾아봐야한다. 이미 AutoInt+ 모델은 초기에 이미 충분한 성능을 보여준 것 같다..!